# Homework 2: Recipe Bot Error Analysis

This notebook walks through finding and categorizing failure modes in a Recipe Bot.

**What you'll learn:**
- How to systematically generate test queries
- How to do open coding to identify patterns
- How to build a failure mode taxonomy

Video walkthrough: https://youtu.be/h9oAAAYnGx4

**Bonus**: [Using AI Assisted Coding to Tackle Homework Problems](https://link.courses.maven.com/c/eJw80M2upCAQBeCngZ0Gil8XLGbja5gCymkTbAyoyX37id2Tu6rUl7OoOqm-ajuXLQeQk554qlfr9OxST8rxHHQWMhpOQTrrrFIgNacdt7Kkgr2H2CrmhP38r-fPQYHerZZCmdP7Xr5-XVsOR6t5hKSzIXKDB2MHnQwNHiQMWhIJQ-C9Q_4K3mmbY1zRC_LZw-Scw9XHKCe_Kot8CyDACimMdFIpNRpjwGaX_JogSeuZFt9_-rjjTe8x1Z1vfVlb3ZePhBlLJ17C6zyPztQfBjOD-TfNYD6wFXwnGgrGzmCmG8szQYAZFIO5_5SC8Xpsr_kq9El5J4ziLWwdMY1rwfPFtPj7VPE54w7wLwAA__8a93gB)

![AI Assisted Coding Walkthrough Location](../imgs/AIHwWalkthrough.png)

In [1]:
import json
from pathlib import Path

## 1. Look at Your Data First

Before writing any code, **look at your data**. We have pre-generated query/response pairs in `reference_files/query_response.jsonl`.

> 💡 **What's a JSONL file?** It's just a text file where each line is a separate JSON object. Great for processing large datasets one record at a time.

### Use the HTML Viewer

Open `reference_files/viewer.html` in your browser and upload the JSONL file. This lets you:
- Navigate between traces with arrow keys
- See the full query and response for each trace
- Get a feel for the data before coding

**Pro tip**: You can vibe-code your own viewer. Try this prompt:

> "Make a self-contained HTML file to view JSONL files. It should let me upload a file, navigate between records, and display all fields nicely."

This is a useful skill - simple viewers help you understand data quickly.

In [2]:
BASE_PATH = Path('reference_files')

# Load query/response pairs (JSONL format)
traces = []
with open(BASE_PATH / 'query_response.jsonl') as f:
    for line in f:
        if line.strip():
            traces.append(json.loads(line))

print(f"Loaded {len(traces)} query/response pairs")
print(f"Example keys: {list(traces[0].keys())}")

Loaded 250 query/response pairs
Example keys: ['id', 'query', 'response']


In [3]:
# Look at one example
example = traces[0]
print(f"ID: {example['id']}")
print(f"\nQuery: {example['query']}")
print(f"\nResponse (first 500 chars):\n{example['response'][:500]}...")

ID: SYN025

Query: Whatcha got for a 30 min eggs and cheese dinner? 🤔

Response (first 500 chars):
Here's a quick and delicious Egg & Cheese Breakfast Sandwich that you can make in about 30 minutes! Perfect for a satisfying dinner or snack with simple ingredients.

**Ingredients:**
- 2 large eggs
- 2 slices of cheese (cheddar, American, or whatever you have)
- 2 slices of bread or a English muffin
- 1 tablespoon butter or oil
- Salt and pepper to taste
- Optional: a slice of ham, bacon, or cooked sausage for extra flavor

**Steps:**
1. **Prepare your ingredients:** If you're adding meat, cook...


## 2. Understand Query Generation

Good test queries come from thinking about **dimensions** that matter for your system. Think of dimensions as the different "knobs" users might turn when asking for help—each combination creates a unique scenario to test.

For a recipe bot, relevant dimensions might include:
- Dietary restrictions (vegan, gluten-free, keto)
- Meal type (breakfast, dinner, snack)
- Time available (quick, elaborate)
- Skill level (beginner, advanced)

> 💡 **Why dimensions?** Instead of randomly thinking up test cases, dimensions help you systematically cover the space of possible user requests. Mix and match values from each dimension to generate diverse, realistic queries.

In [4]:
# Example dimensions for recipe queries
dimensions = {
    "dietary_restriction": ["low carb", "keto", "no seafood", "vegan", "gluten-free"],
    "meal_purpose": ["potluck", "dinner party", "family meal", "quick lunch"],
    "time_available": ["under 15 min", "under 30 min", "1 hour", "all day"],
    "skill_level": ["beginner", "intermediate", "advanced"]
}

print("Query dimensions:")
for dim, values in dimensions.items():
    print(f"  {dim}: {values}")

Query dimensions:
  dietary_restriction: ['low carb', 'keto', 'no seafood', 'vegan', 'gluten-free']
  meal_purpose: ['potluck', 'dinner party', 'family meal', 'quick lunch']
  time_available: ['under 15 min', 'under 30 min', '1 hour', 'all day']
  skill_level: ['beginner', 'intermediate', 'advanced']


In [5]:
# You can use an LLM to generate combinations and turn them into natural queries
# Here's an example prompt:

combination_prompt = """
Generate 10 unique combinations from these dimensions:
- Dietary: low carb, keto, vegan, gluten-free
- Meal: potluck, dinner party, family meal
- Time: under 15 min, under 30 min, 1 hour

Output as a list of combinations.
"""

query_prompt = """
Turn this combination into a natural user query:
- Dietary: vegan
- Meal: dinner party  
- Time: 1 hour

Write 3 different ways a user might ask this.
"""

print("Example prompts for generating queries:")
print("\n--- Combination generation ---")
print(combination_prompt)
print("\n--- Query generation ---")
print(query_prompt)

Example prompts for generating queries:

--- Combination generation ---

Generate 10 unique combinations from these dimensions:
- Dietary: low carb, keto, vegan, gluten-free
- Meal: potluck, dinner party, family meal
- Time: under 15 min, under 30 min, 1 hour

Output as a list of combinations.


--- Query generation ---

Turn this combination into a natural user query:
- Dietary: vegan
- Meal: dinner party  
- Time: 1 hour

Write 3 different ways a user might ask this.




> 📝 **Exercise:** Generating queries from dimensions is left as an exercise. We recommend using AI-assisted coding to do this—see the **Using AI Assisted Coding to Tackle Homework Problems** walkthrough in the Maven student portal for guidance.

## 3. Open Coding

Open coding means reading through traces and noting patterns **without predefined categories**. This is a qualitative research technique borrowed from social science.

Steps:
1. Read each trace carefully
2. Write down anything that seems off or noteworthy
3. Don't try to categorize yet - just observe

> 💡 **The goal here is discovery.** You're building intuition about what can go wrong. Categories come later.

Video: [Open & Axial Coding Walkthrough](https://youtu.be/AKg27L4E0M8)

> 🛠️ **Annotation Tool:** There is an example FastHTML web app for annotating traces with open coding notes and axial coding labels. See `annotation/README.md` for details.  This is just an example, we recommend you build your own instead with AI assisted coding!

In [6]:
# Let's look at a few traces and practice open coding
print("=" * 60)
print("TRACE 1")
print("=" * 60)
t = traces[0]
print(f"Query: {t['query']}")
print(f"\nResponse:\n{t['response'][:800]}...")
print("\n" + "-" * 40)
print("Notes: (what do you observe?)")
print("- Does the response match what was asked?")
print("- Is the time estimate realistic?")
print("- Are serving sizes specified?")
print("- Any missing information?")

TRACE 1
Query: Whatcha got for a 30 min eggs and cheese dinner? 🤔

Response:
Here's a quick and delicious Egg & Cheese Breakfast Sandwich that you can make in about 30 minutes! Perfect for a satisfying dinner or snack with simple ingredients.

**Ingredients:**
- 2 large eggs
- 2 slices of cheese (cheddar, American, or whatever you have)
- 2 slices of bread or a English muffin
- 1 tablespoon butter or oil
- Salt and pepper to taste
- Optional: a slice of ham, bacon, or cooked sausage for extra flavor

**Steps:**
1. **Prepare your ingredients:** If you're adding meat, cook the ham, bacon, or sausage in a skillet over medium heat until browned and cooked through. Set aside.

2. **Toast the bread:** While the meat cooks, toast your bread slices or English muffin in a toaster or on a skillet until golden brown. Keep warm.

3. **Cook the eggs:** In the same skillet, melt...

----------------------------------------
Notes: (what do you observe?)
- Does the response match what was asked?
- Is 

In [7]:
# Browse more traces
for i, t in enumerate(traces[1:4], 2):
    print("=" * 60)
    print(f"TRACE {i}")
    print("=" * 60)
    print(f"Query: {t['query']}")
    print(f"\nResponse (first 400 chars):\n{t['response'][:400]}...")
    print()

TRACE 2
Query: Looking for quick salmon dinner ideas with lemon and herbs pls!

Response (first 400 chars):
Certainly! Here's a quick and flavorful Lemon Herb Salmon recipe that you can enjoy in about 20 minutes.

### Lemon Herb Salmon

**Ingredients:**
- 2 salmon fillets
- 1 lemon (for juice and zest)
- 2 tablespoons olive oil
- 2 garlic cloves (minced)
- 1 teaspoon dried or fresh thyme
- 1 teaspoon dried or fresh dill (or parsley)
- Salt and freshly ground black pepper to taste
- Lemon slices for garn...

TRACE 3
Query: what's a simple recipe using salmon lemon and fresh herbs that I can make quickly?

Response (first 400 chars):
Certainly! Here's a simple and quick recipe for Lemon Herb Salmon:

**Lemon Herb Salmon Fillet**

**Ingredients:**
- 2 salmon fillets
- 1 lemon (zested and juiced)
- 2-3 tablespoons olive oil
- 2 cloves garlic (minced)
- A handful of fresh herbs (such as parsley, dill, or thyme), chopped
- Salt and freshly ground black pepper to taste

**Steps:**

1. **Prehe

## 4. Build a Failure Mode Taxonomy

After open coding, group your observations into categories. This is where you go from "I noticed some weird things" to "Here are the 4-6 ways this system tends to fail."

> 💡 **Why build a taxonomy?** Categories give you tangible things to fix and measure. Instead of a vague sense that "the bot is bad," you have specific failure modes you can target and track improvement on.

For each failure mode, document:
- **Title**: Clear name
- **Definition**: One sentence explaining it
- **Examples**: 1-2 real examples from your data

In [8]:
# Example taxonomy (from reference_files/failure_mode_taxonomy.md)
taxonomy = {
    "Missing Serving Size": {
        "definition": "Bot fails to specify the number of servings or portion sizes in the recipe.",
        "example": "Recipe lists '2 large eggs, 1 cup spinach' without saying how many people this serves."
    },
    "Overcomplicated Simple Recipes": {
        "definition": "Bot provides too many ingredients or steps for what should be simple.",
        "example": "User asks for 'quick egg recipe' but gets 6+ ingredients and multiple optional steps."
    },
    "Inconsistent Time Estimates": {
        "definition": "Bot provides recipes that don't match requested time constraints.",
        "example": "User asks for '30 min recipe' but steps add up to 45+ minutes."
    },
    "Missing Dietary Verification": {
        "definition": "Bot doesn't verify if ingredients actually meet dietary restrictions.",
        "example": "User asks for gluten-free, bot suggests soy sauce without noting it often contains wheat."
    }
}

print("Example Failure Mode Taxonomy")
print("=" * 40)
for name, details in taxonomy.items():
    print(f"\n{name}")
    print(f"  Definition: {details['definition']}")
    print(f"  Example: {details['example']}")

Example Failure Mode Taxonomy

Missing Serving Size
  Definition: Bot fails to specify the number of servings or portion sizes in the recipe.
  Example: Recipe lists '2 large eggs, 1 cup spinach' without saying how many people this serves.

Overcomplicated Simple Recipes
  Definition: Bot provides too many ingredients or steps for what should be simple.
  Example: User asks for 'quick egg recipe' but gets 6+ ingredients and multiple optional steps.

Inconsistent Time Estimates
  Definition: Bot provides recipes that don't match requested time constraints.
  Example: User asks for '30 min recipe' but steps add up to 45+ minutes.

Missing Dietary Verification
  Definition: Bot doesn't verify if ingredients actually meet dietary restrictions.
  Example: User asks for gluten-free, bot suggests soy sauce without noting it often contains wheat.


## 5. Track Failures (Optional)

Create a spreadsheet to track which failure modes appear in each trace.

Columns:
- Trace ID
- User Query
- Open Code Notes
- One column per failure mode (0 or 1)

In [9]:
# Example tracking structure
tracking_example = [
    {
        "trace_id": "SYN025",
        "query": "Whatcha got for a 30 min eggs and cheese dinner?",
        "notes": "Recipe seems doable in 30 min but has many optional steps",
        "missing_serving_size": 0,
        "overcomplicated": 1,
        "inconsistent_time": 0,
        "missing_dietary_check": 0
    },
    {
        "trace_id": "SYN018",
        "query": "Looking for quick salmon dinner ideas with lemon and herbs pls!",
        "notes": "Says 20 min but includes 10 min marination",
        "missing_serving_size": 1,
        "overcomplicated": 0,
        "inconsistent_time": 1,
        "missing_dietary_check": 0
    }
]

print("Example tracking data:")
for row in tracking_example:
    print(f"\n{row['trace_id']}: {row['query'][:50]}...")
    print(f"  Notes: {row['notes']}")
    failures = [k for k, v in row.items() if k not in ['trace_id', 'query', 'notes'] and v == 1]
    print(f"  Failures: {failures if failures else 'None identified'}")

Example tracking data:

SYN025: Whatcha got for a 30 min eggs and cheese dinner?...
  Notes: Recipe seems doable in 30 min but has many optional steps
  Failures: ['overcomplicated']

SYN018: Looking for quick salmon dinner ideas with lemon a...
  Notes: Says 20 min but includes 10 min marination
  Failures: ['missing_serving_size', 'inconsistent_time']


## Summary

**What we covered:**
1. Loading and examining bot traces
2. Generating test queries using dimensions
3. Open coding to identify patterns
4. Building a failure mode taxonomy
5. Tracking failures in a structured way

**Your deliverables:**
- Your own test queries (or use the provided ones)
- A failure mode taxonomy in `reference_files/failure_mode_taxonomy.md`
- Optional: A tracking spreadsheet

**Next steps:**
- In HW3, you'll use these failure modes to build an LLM judge